In [1]:
path_to_lightning = ! echo $PATH_TO_LIGHTNING
lightning_bin_dir = ! echo $LIGHTNING_BIN_DIR

# if "list index out of range", make sure you set where your nodes and lightning bins are
path_to_lightning = path_to_lightning[0]
lightning_bin_dir = lightning_bin_dir[0]

l1 = f'{lightning_bin_dir}/lightning-cli --lightning-dir={path_to_lightning}/l1'
l2 = f'{lightning_bin_dir}/lightning-cli --lightning-dir={path_to_lightning}/l2'

print("l1's lightning-cli is: ", l1)
print("l2's lightning-cli is: ", l2)


l1's lightning-cli is:  /nix/store/0bq00qd0scchy6vasjpqxqg53aijcca3-clightning-v23.11rc1/bin/lightning-cli --lightning-dir=/home/gudnuf/cashu/pyshu_mint/.lightning_nodes/l1
l2's lightning-cli is:  /nix/store/0bq00qd0scchy6vasjpqxqg53aijcca3-clightning-v23.11rc1/bin/lightning-cli --lightning-dir=/home/gudnuf/cashu/pyshu_mint/.lightning_nodes/l2


In [2]:
import json
from coincurve import PrivateKey, PublicKey
from hashlib import sha256
from random import randint
from lib import generate_blinded_messages, construct_inputs

In [3]:
# l1 = "/usr/local/bin/lightning-cli --lightning-dir=/tmp/l1"
# l2 = "/usr/local/bin/lightning-cli --lightning-dir=/tmp/l2"

# mint 7 ecash tokens in exchange for 7 sats
mint_amount = 7

# melt 7 tokens for 7 sats
melt_amount = 7

# Define Keyset

In [4]:
pubkeys = !$l1 cashu-get-keys | jq -r .keysets[0].keys
pubkeys = json.loads(''.join(pubkeys))

In [5]:
privkeys = !$l1 cashu-dev-get-privkeys
privkeys = json.loads(''.join(privkeys))

# Minting and Melting Cashu Tokens

The following describes the path from:

    1. lightning --> ecash (**minting**)
    2. ecash --> lightning (**melting**)

## Minting
A wallet pays a lightning invoice in exchange for signatures from a mint on blinded messages.

### 1. Wallet requests a quote to *mint*

In [6]:
mint_quote = !$l1 -k cashu-quote-mint amount=$mint_amount unit="sat" | jq -r 
mint_quote = json.loads(''.join(mint_quote))

# quote is the unique id for this exchange
mint_id = mint_quote["quote"]
# `request` is a lightning invoice
mint_invoice = mint_quote["request"]

print("quoteId:", mint_id)
print("bolt11: ", mint_invoice)

quoteId: 1a0909732f848817
bolt11:  lnbcrt70n1pj64u27sp56h36j20qzmez92jjzv34lkueuyglfs0956tmm4wk4welq9pwtdaqpp5ygdv25thaz2h5t2x658qn6lnm49sl0erxwf4sc6cul9ez69hzfcsdqsg9hzq6twwehkjcm9xqyjw5qcqp2fp4prldfmug5jngu8e9gqzs6ycs5pspvj7e0e5ajx0mvpjqy9dpqhnqq9qx3qysgqu09cdnzhq6xrz2s4uk64vc93mzh34ze0zv8m8475wulur0ek2usnyxe9qusl93pdn44qxd8wpq2c4qy6enl83ptj6av8fxqny844yqqqxqr3yh


### 2. Wallet pays invoice

In [7]:
!$l2 pay "$mint_invoice" | jq -r .status

complete


### 3. Check mint status

In [8]:
!$l1 cashu-check-mint "$mint_id"

{
   "quote": "1a0909732f848817",
   "request": "lnbcrt70n1pj64u27sp56h36j20qzmez92jjzv34lkueuyglfs0956tmm4wk4welq9pwtdaqpp5ygdv25thaz2h5t2x658qn6lnm49sl0erxwf4sc6cul9ez69hzfcsdqsg9hzq6twwehkjcm9xqyjw5qcqp2fp4prldfmug5jngu8e9gqzs6ycs5pspvj7e0e5ajx0mvpjqy9dpqhnqq9qx3qysgqu09cdnzhq6xrz2s4uk64vc93mzh34ze0zv8m8475wulur0ek2usnyxe9qusl93pdn44qxd8wpq2c4qy6enl83ptj6av8fxqny844yqqqxqr3yh",
   "paid": true,
   "expiry": 1706306526
}


### 4. Wallet requests tokens from mint
- wallet must first generate blinded messages from tokens they want

In [9]:
secrets = [str(randint(0,10000)) for _ in range(3)]
amounts = [1, 2, 4]
b_messages, rs = generate_blinded_messages(secrets, amounts)

#### 4b. wallet sends blinded msgs for  blinded sigs

In [10]:
# lightning-cli -k cashu-mint quote=<str> blinded_messages=<List[blinded_messages]>
mint_command = f'{l1} -k cashu-mint quote=\'{mint_id}\' blinded_messages=\'{json.dumps(b_messages)}\''
blinded_sigs = !$mint_command
blinded_sigs = json.loads(''.join(blinded_sigs))

# blinded_sigs should be a list
if isinstance(blinded_sigs, dict):
    raise AssertionError("{}".format(blinded_sigs))
else:
    print(blinded_sigs)

[{'amount': 1, 'id': '00f775c2e5e81aa3', 'C_': '023ed5d8381efb00ba05b9e01e609708d2855b216626235f66c368d31e29d619a8'}, {'amount': 2, 'id': '00f775c2e5e81aa3', 'C_': '021d807ca675b341ab51b0b0a1b2f242c946a4bc3d452bbff6c6579e7c8391a971'}, {'amount': 4, 'id': '00f775c2e5e81aa3', 'C_': '0216570cde9f0a293f6957ef4d567241cca5e66c8dcefd242b9fbf58d7f8f63302'}]


## Melting

### 5. Wallet generates and invoice for mint to pay
- in this case `l2` will be the wallet

In [11]:
get_invoice_command = f'{l2} invoice {mint_amount * 1000} {str(randint(1, 10000000))} \'a description\''
l2_melt_invoice = !$get_invoice_command
l2_melt_invoice = json.loads(''.join(l2_melt_invoice)).get('bolt11')
l2_melt_invoice

'lnbcrt70n1pj64u2lsp5cy2k4fx4vp0gv7mfh2n66g8ptfech9ydnm5u02nym2jxnynxtaqqpp5mamyraf2fej9wxjysh6vzv2p445udl39j8tc2xttc8x4m4h3pf3qdq4vysxgetnvdexjur5d9hkuxqyjw5qcqp2fp4puyqfcm5ww7zepv4dlvyp3l3xrsuf8lk4edken0e20m0h7mjylmcq9qx3qysgqtfvlgsf39yj97cksh9pt9hlp0j2tqnhwx32eqflqcwnrg62r2dlq8rjcxfa6mwhftxdd3pgu42yvlkumxe83fc5sh662plv5fa9a5gsptlxejm'

### 6. Wallet requests a quote to *melt*

In [12]:
melt_quote_command = f'{l1} -k cashu-quote-melt req={l2_melt_invoice} unit="sat"'
melt_quote = !$melt_quote_command
melt_quote_id = json.loads(''.join(melt_quote)).get("quote")
melt_quote

['{',
 '   "quote": "b26a02b0c698307b",',
 '   "amount": "7msat",',
 '   "fee_reserve": 0,',
 '   "paid": false,',
 '   "expiry": 0',
 '}']

### 7. Wallet sends tokens for the mint to melt

In [13]:
outputs = blinded_sigs
inputs = construct_inputs(outputs, rs, secrets, pubkeys, privkeys)
inputs


'[{"amount": 1, "C": "0371235aca7bd012f8039f8af7b274d7987a2b98255f1997ab4d06c9137995b9fc", "id": "00f775c2e5e81aa3", "secret": "5733"}, {"amount": 2, "C": "0217b804e2c93cf74bb2ee1b8a223013db0019c77437782899fe0ff42978c34846", "id": "00f775c2e5e81aa3", "secret": "4302"}, {"amount": 4, "C": "025f16748933dbd366ed3c13abdd1f9961a0b7581d69fce7b6b37b7e252cd9ab91", "id": "00f775c2e5e81aa3", "secret": "9987"}]'

In [14]:
melt_command = f'{l1} -k cashu-melt quote={melt_quote_id} inputs=\'{inputs}\''
!$melt_command



{
   "paid": true,
   "preimage": "0be1f2e1991f71d5d7c3e795cf14e03117df34c6e3de8e22238f9c167ae31a38"
}


### 6. Check melt status

In [15]:
!$l1 cashu-check-melt $melt_quote_id

{
   "quote": "b26a02b0c698307b",
   "amount": "7msat",
   "fee_reserve": 0,
   "paid": true,
   "expiry": 0
}
